In [1]:
import sys
from os import getenv
import warnings
warnings.filterwarnings("ignore")

GONZAG_DIR   = getenv('HOME')+'/DEV/gonzag'
sys.path.append(GONZAG_DIR)

import gonzag as gzg
from gonzag.config import ldebug

GONZAG_DATA_DIR = '/Users/auraoupa/Work/git/gonzag/gonzag_input'

# Satellite input data:
name_sat = 'Sentinel3A'
file_sat = GONZAG_DATA_DIR+'/SENTINEL3A_20170130-20170303.nc'
name_ssh_sat = 'sla_unfiltered'

# Model input data:
name_mod = 'eNATL60-WestMed'
file_mod = GONZAG_DATA_DIR+'/sossheig_box_WestMed_eNATL60-BLBT02_20170201-20170228.nc'
name_ssh_mod = 'sossheig'
file_lsm_mod = file_mod; name_lsm_mod = '_FillValue' ; # we use _FillValue attribute of "nams_ssh_mod" in "file_mod"
l_griddist = False ; # grid is not strongly distorded





In [2]:
    (it1,it2), (Nts,Ntm) = gzg.GetEpochTimeOverlap( file_sat , file_mod )


In [3]:
print(it1,it2)

1485909000.0 1488324600.0


In [4]:
from sys import exit
from math import radians, cos, sin, asin, sqrt, pi, tan, log, atan2, copysign
import numpy as nmp
from gonzag.config import ldebug, R_eq, R_pl, deg2km
IsZarr=False

In [5]:
ncfile_sat, ncfile_mod = file_sat , file_mod


In [6]:
    if not IsZarr:
        from gonzag.ncio import GetTimeInfo
#        print('import nc')
    else:
        from gonzag.zarrio import GetTimeInfo
#        print('import zarr')
    #
    nts, range_sat = GetTimeInfo( ncfile_sat )
    ntm, range_mod = GetTimeInfo( ncfile_mod )
    #
    (zt1_s,zt2_s) = range_sat
    (zt1_m,zt2_m) = range_mod
    if ldebug: print('\n *** [GetEpochTimeOverlap()] Earliest/latest dates:\n   => for satellite data:',zt1_s,zt2_s,'\n   => for model     data:',zt1_m,zt2_m,'\n')
    if (zt1_m >= zt2_s) or (zt1_s >= zt2_m) or (zt2_m <= zt1_s) or (zt2_s <= zt1_m):
        MsgExit('No time overlap for Model and Track file')


In [7]:
print((max(zt1_s, zt1_m), min(zt2_s, zt2_m)), (nts, ntm))

(1485909000.0, 1488324600.0) (1599082, 672)


In [8]:
print(zt1_s)

1485733479.027518


In [12]:
import numpy as nmp
from netCDF4 import Dataset, num2date, default_fillvals
from calendar import timegm
from datetime import datetime as dtm
from gonzag.config import ldebug, ivrb, rmissval
from gonzag.utils  import MsgExit


In [10]:
ncfile=ncfile_sat

In [14]:
def ToEpochTime( vt, units, calendar ):
    '''
    # INPUT:
    #   * vt: time vector provided as something like: "days since ..."
    #   * units, calendar:
    #
    # OUTPUT:
    #   * vet: time vector converted to UNIX epoch time,
    #          aka "seconds since 1970-01-01 00:00:00"
    #          => as FLOAT !! not INTEGER !!!
    '''
    cfrmt = '%Y-%m-%d %H:%M:%S'
    #
    lvect = not (nmp.shape(vt)==())
    #
    if lvect:
        nt = len(vt)
        t0 = vt[0]
    else:
        t0 = vt
    if ivrb>0: print(' *** [ToEpochTime()]: original t0 as "'+units+'" => ', t0)
    t0d = num2date( t0, units, calendar )
    if ivrb>0: print(' *** [ToEpochTime()]: intitial date in datetime format => ', t0d)

    # We need to round this to the nearest second, because our target format is Epoch time (seconds since 1970)
    # and we want an integer!
    rdec = t0d.microsecond*1.E-6
    # t0 as "float" UNIX time:
    t0E = float( timegm( dtm.strptime( t0d.strftime(cfrmt) , cfrmt ).timetuple() ) + rdec )

    if lvect:
        # we are not going to convert the whole array but instead:
        if   units[0:10] == 'days since':
            vdt = (vt[1:] - vt[0])*86400.
        elif units[0:11] == 'hours since':
            vdt = (vt[1:] - vt[0])*3600.
        elif units[0:13] == 'seconds since':
            vdt =  vt[1:] - vt[0]
        else:
            MsgExit('[ToEpochTime()] => unknown time unit: '+units)
        vet = nmp.zeros(nt)
        vet[0]  = t0E
        vet[1:] = t0E + vdt[:]
        del vdt
    else:
        vet = t0E
    #
    return vet


In [15]:
    if ldebug: print(' *** [GetTimeInfo()] Getting calendar/time info in '+ncfile+' ...')
    id_f = Dataset(ncfile)
    list_dim = id_f.dimensions.keys()
    list_var = id_f.variables.keys()
    for cd in [ 'time', 'time_counter', 'TIME', 'record', 't', 'none' ]:
        if cd in list_dim: break
    if cd == 'none': MsgExit('found no time-record dimension in file '+ncfile)
    if ldebug: print('   => time/record dimension is "'+cd+'"')
    nt = id_f.dimensions[cd].size
    cv = cd ; # ASSUMING VAR == DIM !!! Is it bad???
    if not cv in list_var: MsgExit('name of time variable is different than name of time dimension')
    clndr = id_f.variables[cv]
    dt1 = num2date( clndr[0], clndr.units, clndr.calendar ) ; dt2 = num2date( clndr[nt-1], clndr.units, clndr.calendar )
    rt1 = ToEpochTime( clndr[0],    clndr.units, clndr.calendar )
    rt2 = ToEpochTime( clndr[nt-1], clndr.units, clndr.calendar )    
    id_f.close()
    #
    if ldebug: print('   => first and last time records: ',dt1,'--',dt2,' (UNIX epoch: ', rt1,'--',rt2,')\n')


In [24]:
id_f = Dataset(ncfile)
clndr = id_f.variables[cv]
clndr.calendar

'gregorian'

In [25]:
clndr

<class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    axis: T
    calendar: gregorian
    long_name: Time of measurement
    standard_name: time
    units: days since 1950-01-01 00:00:00
unlimited dimensions: time
current shape = (1599082,)
filling on, default _FillValue of 9.969209968386869e+36 used

In [17]:
dt1

cftime.DatetimeGregorian(2017, 1, 29, 23, 44, 39, 27518)

In [18]:
rt1

1485733479.027518

In [16]:
print(nt, (rt1,rt2))

1599082 (1485733479.027518, 1488671475.516345)
